In [2]:
# Βιβλιοθήκες
import pandas as pd
import numpy as np
import gekko as gk
import seaborn as sns
import xarray as xr
from matplotlib import pyplot as plt
import typing
import itertools
import math
import functools
from pca import pca
from IPython.display import display, HTML
import prince
import networkx
import sklearn
import tensorflow as tf
from tensorflow import keras
import multiprocessing
from imblearn.over_sampling import ADASYN, SMOTE
import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, multilabel_confusion_matrix, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from itertools import product
import arviz as az
import aesara
from factor_analyzer import FactorAnalyzer
import aesara.tensor as at
import scipy
import pymc as pm, pymc
import gpflow
from pymc.sampling.jax import sample_numpyro_nuts
import numpyro
import pickle

2022-12-10 11:51:17.001920: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64:/usr/local/cuda-11.4/lib64:
2022-12-10 11:51:17.002367: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64:/usr/local/cuda-11.4/lib64:
2022-12-10 11:51:17.002378: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
/media/alexander-fyrogenis/Elements/Διδακτορικό/Olive Oil/notebooks/evoos_env/lib/python3.10/

In [2]:
# Γενικές επιλογές εμφάνισης
pd.set_option('display.max_rows',10)
pd.set_option('display.max_columns',30)

numpyro.set_platform("gpu")

figsize=(13,6)
fontsize=10
sns.set(rc={'figure.figsize':(15,10),
           'font.size':5})

In [3]:
# Utilities
from sklearn.preprocessing import StandardScaler

std_scale = lambda df: pd.DataFrame(data = StandardScaler().fit_transform(df), columns = df.columns, index=df.index)

def render_df(df:pd.DataFrame):
    from IPython.display import display, HTML
    display(HTML(df.to_html()))
    return

def full_display(r=None, c=None):
    pd.set_option('display.max_rows', r)
    pd.set_option('display.max_columns',c)
    return

def reset_display():
    pd.set_option('display.max_rows',10)
    pd.set_option('display.max_columns',30)

def full_display_once(df:pd.DataFrame, r=None, c=None):
    full_display(r=r, c=c)
    render_df(df)
    reset_display()

def rowwise_value_counts(df:pd.DataFrame):
    vcounts_df = pd.DataFrame(data = df.apply(lambda x: x.value_counts()).T.stack()).astype(int).T
    vcounts_df.index = ['']
    return vcounts_df

invert_dict = lambda e: {v:k for k, v in e.items()}

def categorical_scatter(X:pd.DataFrame, Y:typing.Optional[pd.DataFrame],cols:int=3,max_rows:int = 3,
                             figsize:tuple[int, int] = (30, 15), xaxis_label_size: int=12,
                             yaxis_label_size:int=12, categorical:typing.Optional[str]='hue')->typing.Optional[plt.figure]:
    '''
        Generate pair-wise scatter plots for a given DataFrame, with optional support for large Datasets
        and categorical variables. Yields a figure with `max_rows x cols` scatterplots per call. When
        `categorical=None` only generates pair-wise scatterplots for the columns of X, else generates all
        combinations of X-column pair and Y columns. When `categorical='hue'` values of the categorical Y
        variable will be depicted color-coded and when `categorical=size` different factors of the Y variable
        will have different sizes instead.
        
        Args:
        -----
        
            - X:pandas.DataFrame := The data to depict. When Y is also specified, X is assumed to be the DataFrame
            of indicator variables
            
            - Y:Optional[pandas.DataFrame] := Optional Dataframe of categorical variables to depict. Ignored if 
            `category` is `None`.
            
            - idx_lvl:int=1 := For multilevel indexed dataframes the lowest level to squash the index on. Must be non-negative
            
            - cols:int=3 := Number of columns for the resulting facet grid plot. Must be non-negative. Defaults to 3
            
            - max_rows:int=3 := Maximum number of rows per batch the generator yields. Defaults to 3 and must be non-negative
            
            - figsize:tuple[int, int] := A `height x width` tuple for the generated plots. Defaults to `(30, 15)`
            
            - xaxis_label_size:int=12 := The size of the x-axis titles for each subplot. Must be non-negative and defaults to 12
            
            - yaxis_label_size:int=12 := The size of the y-axis titles for each subplot. Must be non-negative and defaults to 12
            
            - categorical:Optional[str] := Set the display of the categorical variable. (1) `None` ignores `Y` and only displays X
            pair-wise scatterplots, (2) 'hue' displays the categorical variable of `Y` as color and (3) 'size' displays
            the categorical variable with differently sized points
            
        Returns:
        --------
        
            - fig:matplotlib.pyplot.figure := The figure object, a FacetPlot of scatterplots
            
       
       Raises:
        ------
        
            - WIP
            
            - ValueError 
    '''
    XY = pd.concat([Y, X], axis=1)
    idx_lvl= XY.columns.nlevels-1
    skip_multiindex = lambda df,i , idx_level=idx_lvl: df.columns[i][idx_level] if idx_level else df.columns[i]
    if idx_lvl:
        XY.columns = XY.columns.get_level_values(idx_lvl)
    x_combs = math.comb(X.shape[1], 2)
    MAX_ROWS = max_rows
    
    if categorical is not None:
        plots = x_combs*Y.shape[1]
    else:
        plots = x_combs
    ncols = cols
    size_scaling_factor = None
    total_rows = math.ceil(plots/ncols)
    total_figures = math.ceil(total_rows/MAX_ROWS)
    X_pairs = itertools.combinations(range(X.shape[1]), 2)
    subplots = itertools.product(X_pairs,range(Y.shape[1]) ) if categorical is not None else X_pairs
    exhaustion_sentinel = False
    while True:
        if exhaustion_sentinel: break
        fig, axs = plt.subplots(nrows=MAX_ROWS, ncols=ncols, figsize=figsize)
        ax_indices = itertools.product(range(MAX_ROWS), range(ncols), repeat=1)
        plotslice = itertools.islice(subplots, MAX_ROWS*ncols)
        fig_generator = itertools.zip_longest( ax_indices, plotslice, fillvalue = ((None, None), None) )
        for (axi, axj), e in fig_generator:
            e = tuple(flatten(e))
            if e[0] is not None:
                if categorical == 'size':
                    sns.scatterplot(x=skip_multiindex(X, e[0]), y=skip_multiindex(X, e[1]), data=XY,
                       size=skip_multiindex(Y, e[2]), ax=axs[axi, axj], legend=True)
                elif categorical == 'hue':
                    sns.scatterplot(x=skip_multiindex(X, e[0]), y=skip_multiindex(X, e[1]), data=XY,
                       hue=skip_multiindex(Y, e[2]), ax=axs[axi, axj], legend=True)
                elif categorical is None:
                    sns.scatterplot(x=skip_multiindex(X, e[0]), y=skip_multiindex(X, e[1]), data=XY,
                       ax=axs[axi, axj], legend=True)
                axs[axi, axj].set_xlabel(skip_multiindex(X, e[0]))
                axs[axi, axj].set_ylabel(skip_multiindex(X, e[1]))
                axs[axi, axj].xaxis.label.set_size(xaxis_label_size)
                axs[axi, axj].yaxis.label.set_size(yaxis_label_size)
                
            else:
                axs[axi, axj].axis('off')
                exhaustion_sentinel = True
        yield fig 
    return None



def flatten(xs):
    from collections.abc import Iterable
    for x in xs:
        if isinstance(x, Iterable) and not isinstance(x, (str, bytes)):
            yield from flatten(x)
        else:
            yield x

def tidy_multiindex(df:pd.DataFrame, sep:str="."):
    '''
        Compress a hierarchically indexed dataframe to standardized tidy
        format. A unique sepperator `sep` is used to allow reversal. All
        levels of the index are appended together with a delimeter to allow
        reversals.
        
        Args:
        ----
        
            - df:pandas.DataFrame := A `pandas.DataFrame` hierarchically indexed
            
            - sep:str='_._' := A delimenter delineating the different levels
            of the index. Ensure it is not present in any column name to avoid
            a malformed index
            
        Returns:
        --------
        
            - ndf:pandas.DataFrame := The DataFrame with a single-level index
    '''
    tidy_cols = (functools.reduce(lambda e1,e2: str(e1)+sep+str(e2), col ) for col in df.columns)
    ndf = df.copy(deep=True)
    ndf.columns = tidy_cols
    return ndf

def reverse_tidy_multiindex(df:pd.DataFrame, sep="."):
    '''
        Reverses the tidying to a hierachical format. Different
        levels of the index are identified based on "sep"
        
        Args:
        -----
        
            - df:pandas.DataFrame := The dataframe to process
            
            - sep:str='_._' := The string delimeter, sepperating
            values for different levels of the index
            
        Returns:
        -------
        
            - ndf:pandas.DataFrame := The dataframe with hierarchical index
    '''
    h_cols = (tuple(col.split(sep)) for col in df.columns)
    ndf = df.copy(deep=True)
    ndf.columns = pd.MultiIndex.from_tuples(h_cols)
    return ndf

def undummify(df:pd.DataFrame,cols:list[str, tuple[str]],ncol_name:typing.Union[str,tuple[str]],
              sep:typing.Optional[str]=None,
              rmap:typing.Optional[dict[int, typing.Union[str, tuple[str]]]]=None
             )->pd.DataFrame:
    '''
        Reverses hot-encoded variables in the DataFrame. A series of hot-encoded
        variable levels $(i_1, i2, \dots, i_k)$ is mapped to a single new column
        $(k)$, whose name is specified by `ncol_name`, in the new dataframe. Pre
        vious level columns are dropped.
        
        Args:
        ----
        
            - df:pandas.DataFrame := The DataFrame to operate upon
            
            - cols:list[str, tuple[str]] := A list of columns, representing the
            levels of a categorical variable
            
            - sep:Optional[str] := sepperator for variable level. Currently ignored
            
            - ncol_name:Union[str, tuple[str]] := Name of the new categorical column
            
            - remap:Optional[dict[int, Union[str, tuple[str]]]] := A dictionary mapping
            of categorical levels to values. Keys are the assumed to be levels, values
            are assumed to be values (i.e. strings). When provided, the previous levels
            will be replaced by the specified mappings in the new DataFrame
            
        Returns:
        -------
        
            - ndf:pandas.DataFrame := The processed dataframe
     '''
    _df = coredf.loc[:, cols]
    for i, col in enumerate(cols, 1):
        _df.loc[:, col] = i*_df.loc[:, col]
    ndf = df.copy(deep=True)
    ndf.drop(cols, axis=1, inplace=True)
    ndf[ncol_name] = _df.max(axis=1)
    c1 = df.columns.tolist()
    i = c1.index(cols[0])
    swp = ndf.columns.tolist()[:i-1]+[ndf.columns.tolist()[-1]]+ndf.columns.tolist()[i:-1]
    ndf = ndf.loc[:, swp]
    if rmap is not None:
        ndf = ndf.replace(rmap)
    return ndf
list_difference = lambda l1, l2: [e for e  in l1 if e not in set(l2)]

def corrspace_graph(df:pd.DataFrame):
    '''
        Generate a correlation graph representation of `df` datasets.
        Every node in the resulting graph is a variable and every 
        vertex between two nodes represents the correlation between
        the two variables, the correlation being the verted weight
        
        Args:
        ----
        
            - df:pandas.DataFrame := A dataset to depict
            
        Returns:
            - WIP
    '''
    
    return 


class DictTable(dict):
    '''
        Jupyter utility class that overrides the dicts' defaults
        __repr__ rendering the input dictionary to an HTML table
        for convenient jupyter rendering
    '''
    def _repr_html_(self):
        html = ["<table>"]
        for key, value in self.items():
            html.append("<tr>")
            html.append("<td>{0}</td>".format(key))
            html.append("<td>{0}</td>".format(value))
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)
    
    
def boxplots(df:pd.DataFrame, max_features:int=20, title:str="Boxplot",
             **kwargs)->typing.Generator[plt.Figure, None, None]:
    '''
        Wrapper around `pandas.boxplot` for datasets with a large number
        of features. Returns a generator of boxplots of subsets of the total
        features.
        
        Args:
        -----
        
            - df:pandas.DataFrame := The DataFrame to plot
            
            - max_features:int>0 := The maximum number of features to plot
            in each generated boxplot
            
            - title:str='Boxplot' := The title of the plot. A subplot index
            of the form **(X/Y)**, tracking the current plot will be
            appended prior to rendering
            
            - **kwargs:dict[str:Any] := Keyword arguments to be be forwarded
            to pandas.DataFrame.boxplot
            
        Returns:
        -------
        
            - Generator[plt.Figure, None, None] := A generator yielding boxplots
            of feature subsets
    '''
    nplots = math.ceil(df.shape[1]/max_features)
    for i in range(nplots):
        if (i+1)*max_features<df.shape[1]:
            ndf = df.iloc[:,i*max_features:(i+1)*max_features]
        else:
            ndf = df.iloc[:,i*max_features:df.shape[1]]
        fig = ndf.boxplot(**kwargs)
        fig.set_title(title + " ({current}/{total})".format(current=i+1, total=nplots) )
        yield fig
        
def count_missing_nan(df:pd.DataFrame, axis:int=0):
    '''
        Return a new DataFrame with the counts of missing
        and invalid values across specified axis.
        
        Args:
        -----
        
            - df:pandas.DataFrame := The data
            
            - axis:int=0 := The axis across which missing
            values will be enumerated. Defaults to 0 (show
            missing values in each column)
            
        Returns:
        -------
        
            - missing_df:pd.DataFrame := A DataFrame containing
            counts of missing values
    '''
    ndf = pd.DataFrame(df.isna().sum(axis=axis)).T
    ndf.index = ['']
    return ndf


class PlotPrior:
    '''
        WIP
    '''
    def __init__(self,data, columns:typing.Optional[list[str, tuple[str]]]=None,
                 visible:bool=True, stack:bool=True, show_grid:bool=False, ncols:int=3,
                kde:bool=True):
        self.columns=columns
        self.visible=visible
        self.stack=stack
        self.ncols=ncols
        self.kde=kde
        self.plots={} 
    
    def __prep_data(self, data):
        stacked = data.prior.stack(samples=["chain", "draw"])
        if self.columns is None:
            self.columns = data.columns
        for col in self.columns:
            self.plots[col] = pd.DataFrame(data=stacked[col].values.T)
    
    def __call__(self, data, var:str):
        df = self.plots[var]
        k = df.shape[1]
        n = self.ncols
        m = (k - 1) // n + 1
        fig, axes = plt.subplots(nrows=m, ncols=n, figsize=(n * 5, m * 3))
        for i, (name, col) in enumerate(self.plots[var].items()):
            r, c = i // n, i % n
            ax = axes[r, c]
            col.hist(ax=ax)
            if self.kde:
                ax2 = col.plot.kde(ax=ax, secondary_y=True, title=name)
                ax2.set_ylim(0)
        fig.tight_layout()
        return fig,axs
    
def plot_kernel():
    '''
        Utility method for visualizing GP kernels. Produces a square visualization
        of the selected kernel and all possible combinations of hyperparameters
        
        TODO
        +++++
        
            - Extend this for combination kernels by nesting input hyperparameter
            dicts
            
            - Raise combinatronic explsion error when too many hyperparameter values
            are specified
        
        Args:
        ----
        
            - 
            
        Returns:
        --------
        
            -fig:matplotlib.pyplot.Figure := The generated figure
    '''
    
class MultiLayerPerceptronKernel(pymc.gp.cov.Covariance):
    '''
        Multi layer Perceptron Kernel a.k.a Arcsine Kernel
        Code transplanted from `GPy.kern.src.mlp`. This kernel
        is designed to mimic the highly nonlinear processes 
        of a Neural Network and can be thought of as the limit
        of an A.N.N. with a single input layer as the number
        of neurons in the hidden layer goes to infinity. Then
        the N.N., with normal priors over the weights and biases
        becomes equivalent to a G.P. This Kernel is given by:
        
        .. math::

          k(x,y) = \\sigma^{2}\\frac{2}{\\pi }  \\text{asin} \\left
          ( \\frac{ \\sigma_w^2 x^\\top y+\\sigma_b^2}{\\sqrt{\\sigma_w^2x^\\top x +
          \\sigma_b^2 + 1}\\sqrt{\\sigma_w^2 y^\\top y + \\sigma_b^2 +1}} \\right )
          
        Args:
        -----

            - input_dims:int=1 := The number of input dimentions (columns) used for Covariance
            computations. Defaults to 1 (use the first column only).

            - active_dims:Optional[list[int]]=None := None or array-like specifying which input
            dimentions will be used in computing the covariance matrix. Can be specified as a
            a boolean vector or a vector of indices. Optional. Defaults to None, and the first
            `input_dims` columns are selected.

            - variance:Optional[float] := The output kernel variance scaling parameter, usually
            denoted `\eta`. Defaults to 1.0. Must be positive.

            - weight_variance:Optional[Union[float, np.typing.NDArray[float]]]=1.0. The variance
            of the weight in the A.N.N. Can be specified as either a scalar or a matrix of
            appropriate size (same as the first input matrix). Optional.  Defaults to 1.0.

            - bias_variance:Optional[Union[float, np.typing.NDArray[float]]]=1.0. The variance of
            the biases in the A.N.N. Can be either a scalar or a matrix of appropriate size. When
            computing covariance matrices (single input) must be a N-length vector of biases for a
            `N \times M` input matrix. When computing cross-covariance matrices, must be an
            `N \times N` matrix. Optional.  Defaults to 1.0.

            - ARD:bool=False := A(utomatic)R(elevance)D(etermination) flag. Unused and raises
            an error if switched. Optional. Defaults to False.
        
        
        Returns:
        -------
        
            - K:np.typing.NDArray := When called with the `.full(X)` or the `.full(X, Y)` method,
            computes and returns the covariance or cross-covariance matrices.
            
            - diag(K):typing.NDArray := When called with the `.diag(X)` or `.diag(X, Y)` method
            computes and returns the diagonal of the covariance or cross-covariance matrices
    '''
    four_over_tau = 2./np.pi
    
    def __init__(self,input_dims:int, active_dims:typing.Union[list[int], int]=None,
                 variance:float=1.0,
                 weight_variance:typing.Union[float, np.typing.NDArray]=1.0,
                 bias_variance:typing.Union[float, np.typing.NDArray]=1.0, ARD:bool=False):
        super(MultiLayerPerceptronKernel, self).__init__(input_dims, active_dims=active_dims)
        self.variance = variance
        self.weight_variance = weight_variance
        self.bias_variance = bias_variance
        if ARD:
            raise NotImplementedError("Automatic Relevance Determination, not implemented")
        self.ARD = ARD

    
    def _comp_prod(self, X, X2=None):
        if X2 is None:
            return (at.math.square(X)*self.weight_variance).sum(axis=1)+self.bias_variance
        else:
            return (X*self.weight_variance).dot(X2.T)+self.bias_variance
    
    def diag(self, X):
        """Compute the diagonal of the covariance matrix for X."""
        X_prod = self._comp_prod(X)
        return self.variance*MultiLayerPerceptronKernel.four_over_tau*at.math.arcsin(X_prod/(X_prod+1.))
    
    def full(self, X, X2=None):
        if X2 is None:
            X_denom = at.math.sqrt(self._comp_prod(X)+1.)
            X2_denom = X_denom
            X2 = X
        else:
            X_denom = at.math.sqrt(self._comp_prod(X)+1.)
            X2_denom = at.math.sqrt(self._comp_prod(X2)+1.)
        XTX = self._comp_prod(X,X2)/X_denom[:,None]/X2_denom[None,:]
        return self.variance*MultiLayerPerceptronKernel.four_over_tau*at.math.arcsin(XTX)
    
def advi_inspect_ELBO(fit_data, show:bool=True):
    '''
        Utility method to investigate ADVI fit.
        Returns log-ELBO against iterations.
    '''
    _h_advi_hist = fit_data.hist
    advi_elbo = pd.DataFrame(
        {'$log-ELBO$': -np.log(_h_advi_hist),
         'n': np.arange(_h_advi_hist.shape[0])})
    fig = sns.lineplot(y='$log-ELBO$', x='n', data=advi_elbo)
    if show:
        plt.show()
    return fig

def select_subarray(df:pd.DataFrame, targets:list[tuple[str,str,str]], indicators:list[tuple[str,str,str]],
                   scaleX:bool=True, train_split:bool=False,
                    mappings:typing.Optional[dict[str, typing.Union[str,int, float]]]=None,
                    dummify:bool=False,
                   )->tuple[pd.DataFrame, pd.DataFrame]:
    X = df.loc[:,indicators]
    X = std_scale(X) if scaleX else X
    Y = df.loc[:, targets]
    m = Y.loc[~Y.isna().any(axis=1),:].index.intersection(X.loc[~X.isna().any(axis=1),:].index)
    X = X.loc[m, :]
    Y = Y.loc[m,:]
    Y=Y.replace(mappings) if mappings is not None else Y
    Y = pd.get_dummies(Y) if dummify else Y
    if train_split:
        X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y,test_size=.1 ,random_state=44)
        return X_train, X_test, Y_train, Y_test
    else:
        return X, Y

In [4]:
def tf_model(Xtrain, Xtest, Ytrain, Ytest, loss:str='categorical_crossentropy',
             optimizer:str='adadelta',
             layers:typing.Optional[list[tuple[int, str]]]=None,
             metrics=['accuracy'], summary:bool=True, epochs:int=10,
             factivation:str='sigmoid',batch:int=64, callbacks:list=[],
            verbosity:int=1):
    M = Xtrain.shape[1]
    input_layer = tf.keras.Input(shape=(M,))
    x = input_layer
    for neurons, activation in layers:
        x = tf.keras.layers.Dense(math.ceil(neurons), activation=activation)(x)

    x = tf.keras.layers.Dense(math.ceil(Ytrain.shape[1]), activation=factivation)(x)
    model = tf.keras.models.Model(inputs=input_layer, outputs=x)
    model.compile(optimizer=optimizer, loss=loss, 
                metrics=metrics)
    if summary:
        model.summary()
    model.fit(Xtrain, Ytrain, epochs=epochs, validation_data = (Xtest, Ytest),
              batch_size=batch, callbacks=callbacks, verbose=verbosity)
    return model
# Multiple query utility
query_multiple = lambda df, col, multiquery : functools.reduce(lambda p,n: "{col} == '{p}' | ".format(p=p, col=col) + "{col} == '{n}".format(n=n, col=col), multiquery).split(" == '",1)[1]+"'"

In [5]:
def tf_autoencoder(Xtrain, Xtest, Ytrain, Ytest, loss:str='categorical_crossentropy',
             optimizer:str='adadelta',
             layers:typing.Optional[list[tuple[int, str]]]=None,
             metrics=['accuracy'], summary:bool=True, epochs:int=10,
             activation:str='sigmoid',batch:int=64 ):
    M = X.shape[1]
    input_layer = tf.keras.Input(shape=(M,))
    x = input_layer
    rev_layers = copy(layers)
    rev_layers.reverse()
    for neurons, activation in layers:
        x = tf.keras.layers.Dense(math.ceil(neurons), activation=activation)(x)
    encoder = x
    for neurons, activations in rev_layers:
        x = tf.keras.layers.Dense(math.ceil(neurons), activation=activation)(x)
    x = tf.keras.layers.Dense(math.ceil(Ytrain.shape[1]), activation='sigmoid')(x)
    decoder = x
    model = tf.keras.models.Model(inputs=input_layer, outputs=x)
    model.compile(optimizer=optimizer, loss=loss, 
                metrics=metrics)
    if summary:
        model.summary()
    model.fit(Xtrain, Ytrain, epochs=epochs, validation_data = (Xtest, Ytest),
              batch_size=batch, verbose=1)
    return model

In [3]:
def load_data():
    with open('core_mappings.pickle', 'rb') as handle:
        core_mappings = pickle.load(handle)
    with open('r_core_mappings.pickle', 'rb') as handle:
        r_core_mappings = pickle.load(handle)
    df = pd.read_excel('evoos_processed.xlsx',header=2)
    return core_mappings,r_core_mappings,df
core_mappings, r_core_mappings, df = load_data()

#### ANN με Tensorflow

In [51]:
# from copy import copy
# rlayers = list((math.ceil(e), 'tanh') for e in range(X.shape[1], 2, -10) )
# llayers = copy(rlayers)
# llayers.reverse()
# layers = rlayers+llayers
# layers

In [52]:
# tf_autoencoder(Xtrain, Xtrain, Xtrain, Xtrain, loss='mse', layers=rlayers, epochs=1000)

In [53]:
layers = [(math.ceil(X.shape[1]*1), 'tanh') for _ in range(1,9)]
layers

[(2, 'tanh'),
 (2, 'tanh'),
 (2, 'tanh'),
 (2, 'tanh'),
 (2, 'tanh'),
 (2, 'tanh'),
 (2, 'tanh'),
 (2, 'tanh')]

In [54]:
# from imblearn.over_sampling import ADASYN
# ada = ADASYN(random_state=44)
# Xr, Yr = ADASYN(random_state=44).fit_resample(chem_indicators, origin_targets.replace(r_core_mappings).iloc[:,[0]])
# balancer = {4:22,5:33}

In [55]:
def oversample(X, Y, sampling_strategy:list[float, str]):
    sentinel=True
    Yprime= Y.replace(r_core_mappings)
    for (idx, col), sampling in zip(enumerate(Yprime.columns), sampling_strategy):
        if sampling[1] is None:
            ada = ADASYN(random_state=44, sampling_strategy=sampling[0])
        else:
            ada = ADASYN(random_state=44, sampling_strategy=sampling[0], n_neighbors=sampling[1])
       
        _X, _Y = ada.fit_resample(X, Yprime.iloc[:,[idx]])
        if not sentinel:
            Yr =pd.concat([Yr, _Y], axis=1)
        else:
            Yr = _Y
            Xr = _X
            sentinel=False
    Xr = pd.DataFrame(data = sklearn.decomposition.PCA(n_components = .99).fit_transform(Xr ))
    return Xr, Yr
# x,y=oversample(chem_indicators, origin_targets.iloc[:,[-1]], [
#     (balancer,4), ('minority',None), (0.8,None), ('minority', 4), (0.7, None)])
# Xtrain, Xtest, Ytrain, Ytest = data_prep(x,y, dummify=True)

In [56]:
# Preprocess data, examining location only
# sklearn's and pca packages PCA implementations give conflicting results. Needs investigation
# sklearn should probably be prefeared
Y = pd.get_dummies(tidy_multiindex(origin_targets.replace(r_core_mappings)).astype(int), columns=['origin.cultivation.altitude', 'origin.location.loc', 'origin.production.centr'])
X = std_scale(chem_indicators)

In [57]:
Y = origin_targets
Y=tidy_multiindex(Y.replace(r_core_mappings)).astype(int)
Y = pd.get_dummies(
    Y,
    columns=['origin.cultivation.altitude', 'origin.location.loc', 'origin.production.centr'])
X = std_scale(chem_indicators)
# Adjusting columns to preserve order such that later iloc dicts dont't have to be adjusted and
# cent stays at the end
Y

,origin.cultivation.org_b,origin.cultivation.wat,origin.cultivation.altitude_1,origin.cultivation.altitude_2,origin.cultivation.altitude_3,origin.location.loc_1,origin.location.loc_2,origin.location.loc_3,origin.location.loc_4,origin.location.loc_5,origin.production.centr_2,origin.production.centr_3
0,0,0,0,0,1,1,0,0,0,0,0,1
1,0,1,1,0,0,1,0,0,0,0,1,0
2,0,1,1,0,0,1,0,0,0,0,1,0
3,0,1,1,0,0,1,0,0,0,0,0,1
4,0,0,1,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
530,1,1,1,0,0,0,0,0,0,1,0,1
531,1,0,0,0,1,0,0,0,0,1,0,1
532,1,0,0,0,1,0,0,0,0,1,0,1
533,1,0,0,0,1,0,0,0,0,1,0,1


In [58]:
losses = dict(organic='binary_crossentropy',
              altitude='categorical_crossentropy',
              irregation='binary_crossentropy',
             location='binary_crossentropy',
             centrifuge='categorical_crossentropy'
             )
metrics = dict(
            organic=[
                tf.metrics.BinaryCrossentropy(),
                tf.metrics.BinaryAccuracy()],
            altitude=[
                tf.metrics.CategoricalCrossentropy(),
                tf.metrics.CategoricalAccuracy()
            ],
            irregation=[tf.metrics.BinaryCrossentropy(),
                        tf.metrics.BinaryAccuracy()
                       ],
            location=[tf.metrics.CategoricalCrossentropy(),
                      tf.metrics.CategoricalAccuracy()
                     ],
            centrifuge=[tf.metrics.CategoricalCrossentropy(),
                        tf.metrics.CategoricalAccuracy()
                       ],
             )
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X,Y, random_state=2)

2022-11-27 09:54:44.471769: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-27 09:54:44.473345: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-27 09:54:44.473763: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-11-27 09:54:44.474212: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-11-27 09:54:44.474723: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [59]:
Y_train_partitioned = dict(
        organic=Y_train.iloc[:,[0]],
              altitude=Y_train.iloc[:,2:5],
              irregation=Y_train.iloc[:,[1]],
             location=Y_train.iloc[:,5:10],
             centrifuge=Y_train.iloc[:, 10:],
            )
Y_test_partitioned = dict(
        organic=Y_test.iloc[:,[0]],
              altitude=Y_test.iloc[:,2:5],
              irregation=Y_test.iloc[:,[1]],
             location=Y_test.iloc[:,5:10],
             centrifuge=Y_test.iloc[:, 10:],
)
Y_train_partitioned_values = {k: v.values for k, v in Y_train_partitioned.items()}
Y_test_partitioned_values = {k: v.values for k, v in Y_test_partitioned.items()}

In [60]:
Y_full = dict(
        organic=Y.iloc[:,[0]],
              altitude=Y.iloc[:,2:5],
              irregation=Y.iloc[:,[1]],
             location=Y.iloc[:,5:10],
             centrifuge=Y.iloc[:, 10:],
            )

In [61]:
inputs = tf.keras.layers.Input(shape=(X.shape[1],))
x = inputs
for neurons, activation in layers:
    x = tf.keras.layers.Dense(neurons, activation)(x)
    x = tf.keras.layers.Dropout(0.5)(x)
output_organic = tf.keras.layers.Dense(1, activation = 'sigmoid', name='organic')(x)
output_altitude = tf.keras.layers.Dense(3, activation='softmax', name='altitude')(x)
output_irregation = tf.keras.layers.Dense(1, activation='sigmoid', name='irregation')(x)
output_location = tf.keras.layers.Dense(5, activation='softmax', name='location')(x)
output_centrifuge = tf.keras.layers.Dense(2, activation='softmax', name='centrifuge')(x)
ann_model = tf.keras.Model(inputs=inputs, outputs=[output_organic, output_altitude, output_irregation, output_location,output_centrifuge ], name='origin_model')
ann_model.compile(loss=losses,
              optimizer='adam',
              metrics=metrics)

In [62]:
# log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"_multishot_w_accuracy_equal_8_hidden_dropout_tanh_adam_batch_64"
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
# ann_model.fit(X.values, Y_full, epochs=700, callbacks=[tensorboard_callback],
#              validation_data=(X.values, Y_full))

In [63]:
log_dir = "logs/fit/origin/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"_multishot_w_accuracy_equal_8_hidden_dropout_tanh_adam_batch_64"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
ann_model.fit(X_train.values, Y_train_partitioned_values, epochs=700, callbacks=[tensorboard_callback],
             validation_data=(X_test.values, Y_test_partitioned_values))

Epoch 1/700
12/12 [==============================] - 5s 115ms/step - loss: 3.9302 - organic_loss: 0.7030 - altitude_loss: 1.1179 - irregation_loss: 0.7179 - location_loss: 0.6979 - centrifuge_loss: 0.6935 - organic_binary_crossentropy: 0.7030 - organic_binary_accuracy: 0.5826 - altitude_categorical_crossentropy: 1.1179 - altitude_categorical_accuracy: 0.3361 - irregation_binary_crossentropy: 0.7179 - irregation_binary_accuracy: 0.6975 - location_categorical_crossentropy: 1.6311 - location_categorical_accuracy: 0.6779 - centrifuge_categorical_crossentropy: 0.6935 - centrifuge_categorical_accuracy: 0.5938 - val_loss: 3.9728 - val_organic_loss: 0.6952 - val_altitude_loss: 1.1401 - val_irregation_loss: 0.7158 - val_location_loss: 0.7013 - val_centrifuge_loss: 0.7204 - val_organic_binary_crossentropy: 0.6952 - val_organic_binary_accuracy: 0.5126 - val_altitude_categorical_crossentropy: 1.1401 - val_altitude_categorical_accuracy: 0.2689 - val_irregation_binary_crossentropy: 0.7158 - val_irre

KeyboardInterrupt: 

In [ ]:
# m = tf_model(Xtrain, Xtest, Ytrain, Ytest, factivation = 'sigmoid', 
#         loss='categorical_crossentropy', metrics = ['accuracy'],
#         epochs=4000, layers=layers , ,
#         verbosity=1)

Κατασκευάστηκε νευρωνικό δίκτυο με την εξής αρχιτεκνονική: Τα δεδομένα εισαγωγής είναι οι τιμές των 63 μετρούμενων χημικών παραμέτρων και αυτές αντιστοιχίζονται 63 νευρώνες-εισαγωγής. Μεσολαβεί κάποιος αριθμός κρυφών επιπέδον, ποικήλου αριθμού νευρώνων και καταλήγουμε στο layer δεδομένων-εξαγωγής. Οι μεταβλητές-στόχοι είναι ποικήλων τύπων και αντιπροσωπεύουν πολλαπλούς τύπους προβλημάτων, επομένως το τελικό επίπεδο συντέθηκε από μικρότερες, επί μέρους ομάδες νευρώνων, ως εξής.Για την πρόβλεψη του τύπου καλλιέργειας (οργανική ή συμβατική) χρησιμοποιήθηκε μοναδικός νευρώνας, ως πρόβλημα **δυαδικής κατηγοριοποίησης (*binary classification*)**. Για την πρόβλεψη του υψόμετρου δοκιμάστηκαν δύο προσεγγίσεις: Στη πρώτη περίπτωση, το πρόβλημα αντιμετοπίστηκε ως πρόβλημα πολυταξικής κατηγοριοποίησης, χωρίς αλληλοεπικάλυψης. Τα τα τρία επιμέρους υψόμετρα (Πεδινό, Ορεινό, Ημιορεινό) αντιμετοπίστηκαν ως αμοιβέως αποκλειόμενες κατηγοριές, οπότε κωδικοποιήθηκαν με τρεις νευρώνες εξόδου και συνάρτηση-ενεργοποίησης **softmax**, έτσι τα δεδομένα εξόδου είναι δυάνυσμα μήκους 3, τα στοιχεία του οποίου είναι αριθμοί στο διάστημα $[0,1]$, αθροίζουν στη μονάδα. Τότε η ενεργοποίηση των τριών νευρώνων αντιπροσωπεύει κατανομή πιθανοτήτων επί των τριών υψομέτρων. Μια εναλλακτική προσέγγιση είναι να αντιμετοπίσουμε τη περίπτωση ημιπεδινού υψομέτου ως περίπτωση ταυτόχρονης συμμετοχής στις κατηγορίες Πεδινό και Ημιπεδινό. Αντιπροσωπεύεται με δύο νευρώνες και σιγμοειδή συνάρτηση ενεργοποίησης Για την περίπτωση της άρδευσης, πρόκειται για δυαδική περίπτωση, οποτε χρησιμοποιήθηκε μοναδικό νευρώνας με σιγμοειδή ενεργοποίηση. Για την φυγοκέντριση, μπορούμε να την αντιμετοπήσουμε απλούστερα ως δυαδική περίπτωση (αν δεν είναι τριφασικό, είναι δυφασικό) κάτι που είναι υπολογιστικά απλούστερο, ή ως πολυταξικό περίπτωση που είναι κοντινότερα στη φυσική ερμηνεία. Στη τελευταία περίπτωση χρησιμοποιήθηκαν δύο νευρώνες με ενεργοποίηση softmax. Η γεωγραφική προέλευση είναι πρόβλημα πολυταξικης, classification, χωρίς αλληλοεπικάλυψη, με 5 νευρώνες εξόδου και ενεργοποίηση softmax.

Διερευνήθηκαν διάφορες αρχιτεκτονικές, με διαφορετικό πλήθως νευρώνων και επιπέδων, διαφορετικών ενεργοποιήσεων κρυφών επιπέδων και εξόδων, βελτιστοποιητών και άλλων παραμέτρων. Αρχικά χρησιμοποιήθηκε προσέγγισε βελτιστοποίησης τύπου "αναζήτηση πλέγματος" (*grid search*). Ως προς την διαμόρφωση των δύο αμφίσημων παραμέτρων βρέθηκε οτι η προσέγγιση με το υψόμετρο ως πολυταξική, χωρίς αλληλοεπικάλυψη και η διαμόρφωση της φυγοκέντρισης ως πολυταξική αντί δυαδικής δώσαν τα καλύτερα αποτελέσματα, και είναι κοντινότερα στη φυσική ερμηνεία τως. Ως προς τους βελτιστοποιητές, ο **adadelta** έχει σημαντικά χειρότερη απόδωση στις περισσότερες κατηγορίες, ιδιαίτερα ως προς την ύδρευση και φυγοκέντριση ενώ οι υπόλοιποι **Στοχαστική Κατάβαση Βαθμίδας (*SGD*)**, **Αδαμ** , **Adadelta**, **Adagrad**, και **Διάδωση Τετραγωνική Ρίζας Τετραγώνων (*Root Mean Square Propegation, RMSProp*)** δώσαν παρόμοια απόδωση. Επιλέχθηκε ο Adam διότι έδωσε ελαφρώς καλύτερη ακρίβεια ως προς την πολυταξική κατηγοριοποίηση της γεωγραφικής προέλευσης. Δοκιμάστηκαν διάφορες τιμές εποχής (epoch) εκπαίδευσης στο διάστημα [100, 5000] και βρέθηκε οτι η απώλεια έχει σταθεροποιηθεί ικανοποιητικά για $\approx\ 700$. Ως προς την αρχιτεκτονική, ακόμη και με μικρό αριθμό νευρώνων και επιπέδων πχ $2\ \times\ 20$ παρατηρήθηκε overfitting, με την ακρίβεια στο σύνολο εκπαίδευσης να είναι πολύ μεγαλύτερη αυτής στο σύνολο δοκιμής. Το πρόβλημα αντιμετοπίστηκε με την τεχνική dropoout, όπου σε κάθε επανάληψη και για κάθε κρυφό επίπεδο, κάθε νευρώνας έχει 50% πιθανότητα να απενεργοποιηθεί τεχνιτά. Μικρότερος αριθμός νευρώνων στα κρυφά επίπεδα έδωσε μικρότερη ακρίβεια, ενώ εισαγάγωντας 8 κρυφά επίπεδα βελτιώθηκε η ακρίβεια για τον τύπο καλλιέργειας και την άρδευση, αν και παρατηρήθηκε overfitting με την ακρίβεια εκπαίδευσης να αυξάνει σημαντικά. Δοκιμάστηκαν διάφορες συναρτήσεις ενεργοποίησης για τα ενδιάμεσα επίπεδα, και βρέθηκε οτι η ReLu έδωσε τα χειρότερα αποτελέσματα, η υπερβολική εφαπτωμένη τα καλλύτερα.

Συνολικά η ακρίβεια είναι μέτρεια, της τάξης τους 80-90%, ανάλογα με την παράμετρο. Η μέγιστη ακρίβεια είναι για τη περίπτωση της γεωγραφική προελευσης ($\approx\ 90\%$) ενώ ο τύπος καλλιέργειας και η άρδευση είχαν την μικρότερη ακρίβεια ($\approx\ 80\%$). Δοκιμάστηκε μείωση διαστάσεων δεδομένων εισαγωγής με PCA και δεν βρέθηκαν σημαντικές βελτιώσεις ως προς την ακρίβεια ή τον χρόνο εκπαίδευσης. Επειδή πολλές τάξεις έχουν δυσανάλογα πολλές παρατηρήσεις (πχ >300 δείγματα από τη Λέσβο, μόνο 10 από τους Φούρνους) δοκιμάστηκε επανεκπαίδευση αγνοώντας αυτές τις κατηγορίες (Φούρνοι) και δεν βρέθηκε σημαντική βελτίωση. Δοκιμάστηκαν οι τεχνικές ανισόρροπης εκπαίδευσης με υπερ- και υπό- (SMOTE, ADASYN)  δειγματοληψία, όπου βρέθηκε δραματική μείωση της ακρίβειας. Τα άνω μας οδηγούν στο συμπέρασμα οτι η μειωμένη ακρίβεια ίσως να οφείλεται σε έλλειψη επαρκής πληοροφορίας

### Ποικηλίες

In [ ]:
# Missing value. Move this to an earlier processing step
cultivar_targets = cdf.loc[:,('origin', 'cultivar')].iloc[:,2:]/100
cultivar_targets.iloc[244,-1]=1.0
# Keep samples with full known varieties
Y = cultivar_targets[cultivar_targets.loc[:,'unk']==.0]
X = chem_indicators
# Keep only samples valid to both indicators and targets
keep = Y.index.intersection(X.index)
Y = Y.loc[keep,:]
X = std_scale(X.loc[keep,:])
# Remove unknowns column
Y = Y.iloc[:,:-1]
Y.applymap(lambda e: .0 if e<.1 else e)
Y

Αρχικά εξαιτάζουμε τη ποικηλία ως χωριστό πρόβλημα. Εκπαιδεύτικε Ν.Δ. με τόσους νευρώνες εξόδου όσες και οι πιθανές ποικηλίες. Το αποτέλεσμα κανονικοποιήθηκε με ενεργοποίηση softmax ώστε το δεδομένο-εξόδου του Ν.Δ. είναι μοναδική πρόβλεψη % περιεκτηκότητας από κάθε κατηγορίες. Το Ν.Δ. αξιολογήθηκε με το **Μέσο Τετραγωνικό Σφάλμα (*Mean Squared Error*)**, όπου βρέθηκε $\approx\ 2\%$ σφάλμα χωρίς ιδιαίτερη βελτιστοποίηση.

In [ ]:
layers = [(math.ceil(X.shape[1]*1), 'tanh') for _ in range(1,8)]
layers

In [ ]:
losses = 'mse'
metrics = 'mse'

In [ ]:
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X,Y, random_state=2)

In [ ]:
inputs = tf.keras.layers.Input(shape=(X.shape[1],))
x = inputs
for neurons, activation in layers:
    x = tf.keras.layers.Dense(neurons, activation)(x)
    x = tf.keras.layers.Dropout(0.5)(x)
output = keras.layers.Dense(Y.shape[1], activation='softmax')(x)
cultivar_model = tf.keras.Model(inputs=inputs, outputs=output, name='cultivar_model')
cultivar_model.compile(loss=losses,
              optimizer='adam',
              metrics=metrics)

In [ ]:
log_dir = "logs/fit/cultivation/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"_cultivars_sigmoid"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
cultivar_model.fit(X_train.values, Y_train, epochs=1000, callbacks=[tensorboard_callback],
             validation_data=(X_test.values, Y_test))

In [ ]:
# Select all unknown varieties with available chemical data
Xpred = chem_indicators.loc[chem_indicators.index.intersection(cultivar_targets[cultivar_targets.loc[:,'unk']!=.0].index),:]
Xpred

Πειραματικά, μπορούμε να προβλέψουμε τη ποικηλία των κενών κατηγοριών ενδιαφέροντως. Για τα δείγματα για τα οποία έχουμε επαρκείς χημικές παραμέτρους διαθέσιμες και μερικώς ή ολικών άγνωστη ποικηλία έγινε πρόβλεψη της ποικηλίας με το άνω Ν. Δ. Αν και δεν μπορούν να διασταυρωθούν, τα αποτελέσματα είναι αληθοφανοι. Κάθε τιμή κάτω του 10% θεωρήθηκε πρακτικώς μηδέν

In [ ]:
# Predicted samples
## Remember to preserve the index
cultivar_imputed= pd.DataFrame(data=cultivar_model.predict(Xpred), columns=Y.columns, index = Xpred.index)
cultivar_inputed

In [ ]:
cultivar_imputed.applymap(lambda e: .0 if e<.02 else round(e*100))

Βασικό πρόβλημα είναι ελλειπεί δεδομένα σε πάρα πολλά δείγματα. Θα μπορούσαμε να επιχειρήσουμε να αντιμετοπίσουμε αυτό το πρόβλημα με μια ιδιαίτερη μορφή *imputation*. Θα εκαιδεύσουμε μικρότερα μοντέλα για διάφορα υποσύνολα μεταβλητών στόχων και βάση αυτών θα προβλέψουμε τους αντίστοιχουν στόχους. Το αποτέλεσμα θα είναι συνθετικά ενισχυμένο σύνολο δεδομένο με το οποίο μπορούμε να εκπαιδεύσουμε καθολικό δίκτυο. Μπορούμε να συγκρίνουμε τα αποτελέσματα με αυτά άλλων προσεγγίσεων imputation και χωρίς καθόλου imputation

### Ωρίμανση, Διαδικασία Ελαιοποίησης

In [42]:
production_targets = coredf.loc[:, ('origin', 'production')]
# full_display_once(production_targets.iloc[:,:])
nan=production_targets.iloc[:].isna().any(axis=1)
production_targets[~nan]

/tmp/ipykernel_30418/1985942444.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  production_targets = coredf.loc[:, ('origin', 'production')]


,mat_null,mat_pa,mat_pi,mat_p,mat_m,doil,colmeth_eld,colmeth_eldi,colmeth_mhx,colmeth_temp,colmeth_dixt,colmeth_xte,centr,watadd_b,kbtemp,kntemp,stemp
0,0.0,1.0,1.0,0.0,1.0,10.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,70,34.0,36.0
1,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,60,26.0,27.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,90,25.0,28.0
3,0.0,1.0,1.0,0.0,1.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,1.0,70,34.0,34.0
4,0.0,1.0,1.0,0.0,1.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,90,36.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,0.0,0.0,1.0,0.0,1.0,6.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,1.0,40,29.0,27.0
436,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,1.0,30,30.0,28.0
447,0.0,0.0,0.0,0.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,20,25.0,30.0
448,0.0,0.0,1.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,20,25.0,30.0


Ως προς τις υπόλοιπες παραμέτρους παραγωγής, το βασικό μας πρόβλημα είναι οτι υπάρχουν πάρα πολλές κενές τιμές ως προς τη μέθοδο συλλογής. Την αφαιρούμε προσωρινά

In [65]:
production_no_colmeth = pd.concat([production_targets.iloc[:,:6],production_targets.iloc[:,12:] ], axis=1)
# Drop all nans and discard the first column as it tracks nan. Discard centr column as it's been modeled previously
production_no_colmeth_no_nan = production_no_colmeth[~production_no_colmeth.isna().any(axis=1)].iloc[:,1:].drop(['centr'], axis=1)
production_no_colmeth_no_nan
# Scale inputs
X = tidy_multiindex(chem_indicators)
Y = production_no_colmeth_no_nan
# Select all samples with valid entries for both predictors and targets
valid = Y.index.intersection(X.index)
X, Y= X.loc[valid,:], Y.loc[valid,:]
# Rearrange Y columns to group variables by problem type
Y = pd.concat(
    [Y.iloc[:, :4], Y.iloc[:,6:], Y.iloc[:,[4]], Y.iloc[:,[5]]
    ],axis=1)
# Illicit values recorded as 40-50. Replaced with their mean
Y = Y.replace(dict(kbtemp={'40-50':45.0})).astype(float)
# Keep this for later
Xraw=X
X = pd.DataFrame(data=sklearn.preprocessing.MinMaxScaler(feature_range=(-1,1)).fit_transform(X),
                columns=X.columns, index=X.index)

In [ ]:
# Split training data
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, random_state=44)

Η ποικηλία είναι πολυταξικό πρόβλημα με αλληλοεπικάλυψη (**sigmoid**), η προσθήκη νερού δυαδική ταξινόμηση και οι θερμοκρασία ελαιοποίησης, θερμοκρασία μάλαξης, ο χρόνος μάλαξης και ο χρόνος ελαιοποίησης είναι προβλήμστα παλινδρόμησης (γραμμική ενεργοποίηση).

$$
\begin{array}{cc}
\mathcal{L}(y_i, \hat{y_i})\ =\ \Bigg[\ \frac 1N\ \sum_{i=1}^N (y_i-\hat{y_i})^2\ \Bigg]\ +\ k^2\Bigg[\ \frac 1N\ \sum_{i=1}^N \Big(\log\big({y_i+\epsilon}\big)-\log\big({\hat{y_i+\epsilon}\big)}\Big)^2\ \Bigg]\ &,k\in \mathbb{R},\ \epsilon\in\mathbb{R}^+
\end{array}
$$

In [ ]:
def log_enhanced_mse(y_true, y_pred, k=5.0, epsilon=1e-5):
    '''
        MSE loss function that boosts small differences in inputs for increased variance.
        
        $$
            \mathcal{L}(y_i, \hat{y_i})\ =\ \Bigg[\ \frac 1N\ \sum_{i=1}^N (y_i-\hat{y_i})^2\ \Bigg]\ +\ k^2\Bigg[\ \frac 1N\ \sum_{i=1}^N (\log{y_i}-\log{\hat{y_i}})^2\ \Bigg]\ \ \ ,k\in \mathbb{R}
        $$
    '''
    mse_loss = tf.keras.backend.mean(tf.math.squared_difference(y_true,y_pred), axis=-1)
    # Computing log(x) via change of base because tf only offers ln(x) and log2(x)
    log_loss = tf.keras.backend.mean(tf.math.squared_difference(
        tf.math.divide(tf.math.log(y_true+epsilon),tf.math.log(10.0)),
        tf.math.divide(tf.math.log(y_pred+epsilon), tf.math.log(10.0))
                   ), axis=-1)
    return mse_loss+tf.math.pow(k,2)*log_loss

In [ ]:
alpha = 1.0
layers = [(math.ceil(X.shape[1]*alpha), 'tanh') for _ in range(1,11)]
maturation_layers = [(math.ceil(X.shape[1]*alpha), 'tanh') for _ in range(1,10)]
regression_layers = [(math.ceil(X.shape[1]*alpha), 'tanh') for _ in range(1,10)]
water_addition_layers = [(math.ceil(X.shape[1]*alpha), 'tanh') for _ in range(1,10)]
DROPOUT_LAYERS=False
DROPOUT_TOLERANCE = 0.3

In [ ]:
# The model's general architecture
inputl = keras.layers.Input(shape=(X.shape[1],) )
x = inputl
for neurons, activation_func in layers:
    x = keras.layers.Dense(neurons, activation=activation_func)(x)
    if DROPOUT_LAYERS:
        x = keras.layers.Dropout(DROPOUT_TOLERANCE)(x)
# Add more neurons for maturation
m = x
for neurons, activation_func in maturation_layers:
    m = keras.layers.Dense(neurons, activation=activation_func)(m)
    # if DROPOUT_LAYERS:
    #     x = keras.layers.Dropout(DROPOUT_TOLERANCE)(x)
# Water addition layers
wa = x
for neurons, activation_func in water_addition_layers:
    wa = keras.layers.Dense(neurons, activation=activation_func)(wa)
    # if DROPOUT_LAYERS:
    #     x = keras.layers.Dropout(DROPOUT_TOLERANCE)(x)
# Extra Regression layers
r= x
for neurons, activation_func in regression_layers:
    r = keras.layers.Dense(neurons, activation=activation_func)(r)
    if DROPOUT_LAYERS:
        r = keras.layers.Dropout(0.6)(r)
maturation_output = keras.layers.Dense(4, activation='sigmoid', name='maturation')(m) # Multiclass, multilabel classification
continuous_output = keras.layers.Dense(4, name='continuous')(r) # Continuous regression
water_addition_output = keras.layers.Dense(1, activation='sigmoid', name='water_addition')(wa) # Binary classification

In [ ]:
# Parametrize training
losses = dict(
    maturation=[keras.losses.BinaryCrossentropy()],
    continuous=[functools.partial(log_enhanced_mse, k=5.0, epsilon=1e+9)],
    water_addition=[keras.losses.BinaryCrossentropy()],
)
metrics = dict(
    maturation=[keras.metrics.BinaryCrossentropy(),
               keras.metrics.BinaryAccuracy()],
    continuous=[keras.metrics.MeanSquaredError(),
               keras.metrics.RootMeanSquaredError()],
    water_addition=[keras.metrics.BinaryCrossentropy(),
                   keras.metrics.BinaryAccuracy()],)

OPTIMIZER = keras.optimizers.RMSprop()
EPOCHS = 2000
BATCH_SIZE = 64

In [ ]:
# It would be usefull to write a range-cutting class for convenience
## Turn a dictionary of pandas.DataFrame to a dictionary of numpy arrays for training
numpize_modeldict = lambda e: {k:v.values for k, v in e.items()}
## Assemble an output dictionary by manual cutting
Y_train_partitioned = dict(
    maturation=Y_train.iloc[:,:4],
    continuous=Y_train.iloc[:,4:8],
    water_addition=Y_train.iloc[:,[-1]],
)
Y_test_partitioned = dict(
    maturation=Y_test.iloc[:,:4],
    continuous=Y_test.iloc[:,4:8],
    water_addition=Y_test.iloc[:,[-1]],
)

In [ ]:
production_model = keras.Model(name='production_model', inputs=inputl,
                               outputs = [maturation_output, continuous_output, water_addition_output])
production_model.compile(loss=losses, optimizer = OPTIMIZER, metrics=metrics)

In [ ]:
log_dir = "logs/fit/production/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"_production_model_rmsprop_8_hidden_1000_epochs_sigmoid_corrected_extra_layers_binary_maturation_no_dropout_log_loss_many_extra_layers_minmax_scaled"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
production_model.fit(X_train.values,numpize_modeldict(Y_train_partitioned),epochs=EPOCHS,
                     batch_size=BATCH_SIZE, validation_data=(X_test.values, numpize_modeldict(Y_test_partitioned)),
                    callbacks=[tensorboard_callback])

In [ ]:
production_new = production_no_colmeth[production_no_colmeth.isna().any(axis=1)].iloc[:,1:].drop(['centr'], axis=1)
Xnew=chem_indicators.loc[chem_indicators.index.intersection(production_new.index),:]
Xnew = pd.DataFrame(data=sklearn.preprocessing.MinMaxScaler(feature_range=(-1,1)).fit_transform(Xnew),
                   columns=Xnew.columns, index=Xnew.index)
Xnew

In [ ]:
production_imputed = pd.DataFrame(data= np.concatenate(production_model.predict(Xnew), axis=1), 
                                 index = Xnew.index, columns=Y_train.columns)
production_imputed.round(2)

In [ ]:
# Προβλέπει με τον μέσο όρο
pd.DataFrame(data=Y_train.mean(axis=0)).T